In [ ]:
# STEP 0. 필요한 패키지 재설치 (openai 구버전 포함)
!pip uninstall -y openai -q
!pip install openai==0.28 -q
!pip install -q transformers pandas openpyxl pillow bert_score

In [ ]:
# STEP 1. 라이브러리 임포트
import os, glob
import openai
import torch
import pandas as pd
from PIL import Image, ImageFile
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
from bert_score import score as bertscore

In [ ]:
# STEP 2. 이미지 로딩 오류 허용
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# STEP 3. OpenAI API 키 설정
openai.api_key = "sk-*****"  # ← 여기에 본인의 OpenAI API 키 입력

In [ ]:
# STEP 4. BLIP-2 모델 로딩
blip_processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")
blip_model = InstructBlipForConditionalGeneration.from_pretrained(
    "Salesforce/instructblip-vicuna-7b", torch_dtype=torch.float16, device_map="auto"
)

In [ ]:
# STEP 5. 이미지 및 정답 엑셀 로딩
excel_path = "/content/fewshot_data.xlsx"
df = pd.read_excel(excel_path)
answer_cols = ["answer_1"]
image_paths = sorted(glob.glob("/content/*.[jpJP][pnNP]*[gG]"))

In [ ]:
# STEP 6. 보조 함수
def generate_caption(image_path):
    image = Image.open(image_path).convert("RGB").resize((384, 384))
    prompt = "You are a helpful assistant. Describe this image in detail."
    inputs = blip_processor(images=image, text=prompt, return_tensors="pt").to("cuda")
    output = blip_model.generate(**inputs)
    caption = blip_processor.batch_decode(output, skip_special_tokens=True)[0].strip()
    return caption.replace(prompt, "").strip()

def generate_gpt4_response(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=150
    )
    return response.choices[0].message["content"].strip()

In [ ]:
# STEP 7. 이미지별 응답 생성
captions, gpt_responses, answer_lists = [], [], []

for idx, image_path in enumerate(image_paths):
    try:
        caption = generate_caption(image_path)
        captions.append(caption)

        prompt = f"{caption}\nAnswer:"
        response = generate_gpt4_response(prompt)
        gpt_responses.append(response)

        row_answers = [df.loc[idx, col] for col in answer_cols if pd.notna(df.loc[idx, col]) and str(df.loc[idx, col]).strip()]
        answer_lists.append(row_answers if row_answers else ["[EMPTY]"])

        print(f"✅ {os.path.basename(image_path)} 완료")
    except Exception as e:
        print(f"❌ {os.path.basename(image_path)} 실패: {e}")
        captions.append("[ERROR]")
        gpt_responses.append(f"[ERROR] {e}")
        answer_lists.append(["[EMPTY]"])

In [ ]:
# STEP 8. BERTScore 평가
best_scores = {
    "precision": [], "recall": [], "f1": [], "matched_answer": []
}

for refs, cand in zip(answer_lists, gpt_responses):
    if not refs or cand.strip() == "":
        best_scores["precision"].append(0.0)
        best_scores["recall"].append(0.0)
        best_scores["f1"].append(0.0)
        best_scores["matched_answer"].append("[EMPTY]")
        continue

    P, R, F1 = bertscore(cands=[cand]*len(refs), refs=refs, lang="ko", verbose=False)
    best_idx = F1.argmax().item()
    best_scores["precision"].append(P[best_idx].item())
    best_scores["recall"].append(R[best_idx].item())
    best_scores["f1"].append(F1[best_idx].item())
    best_scores["matched_answer"].append(refs[best_idx])

In [ ]:
# STEP 9. 결과 저장
results_df = pd.DataFrame({
    "image": [os.path.basename(p) for p in image_paths],
    "caption": captions,
    "gpt4_response": gpt_responses,
    "matched_answer": best_scores["matched_answer"],
    "bert_precision": best_scores["precision"],
    "bert_recall": best_scores["recall"],
    "bert_f1": best_scores["f1"]
})
results_df.to_excel("/content/llm_result_gpt4.xlsx", index=False)
print("✅ 저장 완료: /content/llm_result_gpt4.xlsx")